### Dynamic Programming for Policy evaluation and improvement 

Implementation is based on the OpenAI gym 

#### Policy Evaluation --> Bellman Expectation Equation
-  Given a policy, evaluate how good a policy is by computing the sum of rewards across all states based on the action from the policy, multiplied with the probability of going there  
-  Iterate the process multiple times until convergence because our initial reward could be wrong. So we need to calculate multiple times until the actual correct values stop changing. 
-  **Notice** that this evaluation procedure will not generate the maximum state-value function
$$
V^\pi(s) =  \sum_{s'} P(s' | s, a) \left[ R(s, a, s') + \gamma V^\pi(s') \right]
$$


#### Policy Iteration ---> Bellman Optimality Equation 
-  Tries to find the optimal policy (like value iteration method)
-  Split calculating state-value function V and policy calculations 
-  **Policy Improvement + policy evaluation**
-  Computes the value function then update the policy based on the value function. Iteratively doing this step until policy stops changing 

#### Value Iteration --> Bellman Optimality Equation  
-  Try to find the optimal policy through iteratively improving the state-value function until convergence 
-  Computes the value function and stores the corresponding actions at each state. Once convergence, we get the policy from the actions at each state 
-  Our sate value function is dependent on the max value generated from a certain action
$$
V^\star(s) = \max_{a}  \sum_{s'} P(s' | s, a) \left[ R(s, a, s') + \gamma V^\star(s') \right]
$$
$$
\pi^\star(s) = \argmax_{a}  \sum_{s'} P(s' | s, a) \left[ R(s, a, s') + \gamma V^\star(s') \right]
$$

In [1]:
import gym 
import torch
import numpy as np

ModuleNotFoundError: No module named 'gym'

In [3]:
env= gym.make("FrozenLake-v1")

In [ ]:
env.reset() 
env.render() 

# the state number is treated as an index on this 4x4 grid, 0-indexed 

'''  
SFFF            # (S: start)
    FHFH            # (F: frozen, safe)
    FFFH            # (H: hole, failure)
    HFFG            # (G: goal)
'''

'  \nSFFF            # (S: start)\n    FHFH            # (F: frozen, safe)\n    FFFH            # (H: hole, failure)\n    HFFG            # (G: goal)\n'

In [6]:
num_states= env.observation_space.n 
num_actions= env.action_space.n
print(f"states: {num_states}, actions: {num_actions}")

states: 16, actions: 4


In [ ]:
state= env.reset()  # get the current state 
print(state)    # 100% probability to be started at that state 

(0, {'prob': 1})


In [ ]:
# actions: LEFT(0), DOWN(1), UP(2) and RIGHT(3)

new_state, reward, is_done, _, info = env.step(3)   # make an action on the right 

In [ ]:
print(new_state)
print(reward)   # the reward is 0 because we landed on the hole 
print(is_done)  # we fell into the hole 

5
0
True


In [27]:
# reset the environment 
state= env.reset()
print(state)

(0, {'prob': 1})


In [ ]:
new_state, reward, is_done, _, info= env.step(1)    # make a step towards the bottom 
print(new_state)
print(info) # returns the transition probability of going to that state 

4
{'prob': 0.3333333333333333}


Transition Probability is stored in a transition matrix, which covers the probability of going to a state s_t from the state s   


In [32]:
from pprint import pprint 
pprint(env.env.P[0])    # prin the transition matrix at state 0 

# the key is the action taken 
# first value is probability of going to that state, 
# second value is the resulting state 
# third value is the reward 
# fourth value is whether the episode has ended 

{0: [(0.3333333333333333, 0, 0.0, False),
     (0.3333333333333333, 0, 0.0, False),
     (0.3333333333333333, 4, 0.0, False)],
 1: [(0.3333333333333333, 0, 0.0, False),
     (0.3333333333333333, 4, 0.0, False),
     (0.3333333333333333, 1, 0.0, False)],
 2: [(0.3333333333333333, 4, 0.0, False),
     (0.3333333333333333, 1, 0.0, False),
     (0.3333333333333333, 0, 0.0, False)],
 3: [(0.3333333333333333, 1, 0.0, False),
     (0.3333333333333333, 0, 0.0, False),
     (0.3333333333333333, 0, 0.0, False)]}


#### Writing policy evaluation functions   
V function is written as  (probability of doing this action) * (reward of the new state + gamma * V of the new state)  

-  Note that we take pi[at | st] as the probability of doing this action 

In [ ]:
threshold= 0.0001    # value required for convergence 
gamma= 0.99         # discounted value gamma 

In [ ]:
# evaluate the policy -> V function 
# we need to find the maximum return from actions available at each state (state-action value )
# to do this, we will need to keep track of the return from each state, so we can use (like a prefix sum)
# DP comes in from storing these values 
def policy_eval (env, policy, gamma, threshold):
    num_states= env.observation_space.n # get number of states 
    V= torch.zeros(num_states)  # V function array 
    max_delta= threshold+1  # \delta which is the maximum differnece between the return of two state action values (check for conv.)
    while max_delta> threshold: 
        temp= torch.zeros(num_states)   # temperary array to store the biggest values as we go through the actions
        for state in range (num_states):    # go through all the possible states (each state is numbered from 0-5, so we can traverse like that)
            action= policy[state].item()    # get the action from our policy at each state
            for proba, new_state, reward, _ in env.env.P[state][action]:    # retrive the information at a given state and apply a given function (from the transition matrix)
                temp[state]+=proba* (reward + gamma * V[new_state]) # compute the temp V value
        max_delta= torch.max(torch.abs(V-temp)) # get the difference of the old value and the new value adn get the difference
        V= temp.clone()     # make V into the temp 
    return V    # when the difference between new policy actions and the original is smaller than convergence term, our policy have converged  

#### Policy Improvement

Finding the best action to take at each state inside the policy function, so that we get the most reward   

Go through all the states, for each action, simulate to get the corresponding reward, add them up and find the best action for each state the policy might face. 

In [ ]:
def policy_improvement (env, V, gamma):
    num_states= env.observation_space.n
    num_actions= env.action_space.n # number of possible actions at each state 
    policy= torch.zeors (num_states)    # our policy stores an action at each state (we want to optimize this)
    for state in range (num_states):
        actions= torch.zeros(num_actions)   # a table of actions to keep in track the reward from each action, so we can get the most optimal action
        for action in range(num_actions):   # iterate through each action to compute the reward
            # evaluate the reward 
            for proba, new_state, reward, _ in env.env.P[state][action]: 
                actions[action] +=proba * (reward + gamma* V[new_state])
        policy[state]= torch.argmax(actions)
    return policy 

**Notice** that in both policy evaluation and improvement function, we add up the temp function from each state, this is the DP/summation part,because we need to sump up all the values from these states 

#### Policy Iteration   

Improves the policy

In [ ]:
def policy_iteration (env, gamma=gamma, threshold= threshold):
    num_states= env.observation_space.n 
    num_actions= env.action_space.n 
    policy= torch.randint(low=0, high=num_actions,size=(num_actions,)).float() # create random actions for initial policy
    while True: 
        V=policy_eval(env, policy, gamma=gamma, threshold=threshold)
        new_policy= policy_improvement(env, V,gamma=gamma)
        if torch.equal(new_policy, policy):
            return V, new_policy
        policy= new_policy.clone() 

### Value Iteration  
Finds the optimal reward at each state 

In [34]:
def value_iteration(env, gamma=0.99, thres = 0.0001):
    num_states = env.observation_space.n
    num_actions = env.action_space.n
    V = torch.zeros(num_states)
    max_delta = thres + 1
    while max_delta > thres:
        temp = torch.zeros(num_states)
        for state in range(num_states):
            v_actions = torch.zeros(num_actions)
            for action in range(num_actions):
                for proba, new_state, reward, is_done in env.env.P[state][action]:
                    v_actions[action] += proba * (reward + gamma * V[new_state])    # Value iteration 
            temp[state] = torch.max(v_actions)              # Select the action with the highest reward
        max_delta = torch.max(torch.abs(V - temp))
        V = temp.clone()
    return V

### Find the optimal policy 

In [ ]:
def extract_optimal_policy(env, V, gamma=0.99):
    num_states, num_actions = env.observation_space.n, env.action_space.n
    optimal_policy = torch.zeros(num_states)
    for state in range(num_states):
        v_actions = torch.zeros(num_actions)
        for action in range(num_actions):
            for proba, new_state, reward, _ in env.env.P[state][action]:
                v_actions[action] += proba * (reward + gamma * V[new_state])
        optimal_policy[state] = torch.argmax(v_actions)
    return optimal_policy